### Install (if needed) all prereqs below

In [ ]:
#!pip install requests
#!pip install openai
#!pip install dotenv
#!pip install pdfminer
#!pip install pdfminer.six

### GPT Configs & Pre-reqs

GPT-4o: 
- Get the GPT-4o key and the endpoint in Azure AI Studio and replace the values in env.env file

In [26]:
import requests
import base64
import json
from openai import AzureOpenAI
from dotenv import dotenv_values
from mimetypes import guess_type

# Create the env file based on the template.env provided. Put all key values, endpoints, etc. according to the template
env_name = "env.env" 
config = dotenv_values(env_name)

# Use your own keys in env.env file for openai_api_key_vision and openai_completion_endpoint_vision parameters

GPT4V_KEY = config['openai_api_key_4o']
GPT4V_ENDPOINT = config['openai_chat_endpoint_4o']

# Provide the PDF path
pdf_path = "files\Wine-Purchase-Order-Template-TemplateLab-pversion.pdf"

### Python code to extract information from PDF and put in plain text

In [27]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""  # If the PDF is password-protected, provide the password here
    maxpages = 0  # Set to 0 to extract all pages
    caching = True
    pagenos = set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password, caching=caching):
        interpreter.process_page(page)

    text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()

    return text

extracted_text = convert_pdf_to_txt(pdf_path)
print(extracted_text)

PURCHASE ORDER

To:
John Smith
Cellar Selections
686 Harrison Lane
Union City - CA - 94587
555 111 5533

P/O NUMBER

679133

Ship To:
Robert Web
RW Wines
7310 Somerset St.
Salinas - CA - 93905
444 222 4466

SHIP VIA

FedEx Service

DATE
19/05/2024

CLIENT ID
12058

TERMS

NET 7

VINTAGE

VARIETAL

2010

2002

2019

2016

2020

2016

2019

Cabernet Sauvignon

Cabernet Sauvignon

Rose

Syrah

Cabernet Reserve

Zinfandel

Merlot

QTY

8

4

24

12

5

10

5

BOTTLE
PRICE

32.00$

45.00$

25.00$

34.00$

26.00$

33.00$

40.00$

DISCOUNT*

TOTAL

5%

5%

15%

15%

5%

5%

5%

243.20$

171.00$

510.00$

346.80$

123.50$

313.50$

190.00$

*) Discounts are granted for the following categories:

10% CASE DISCOUNT (12 PACK)

5% WINE CLUB MEMBERSHIP

1-5% SPECIAL AGREEMENT CLAUSE

Subtotal

Sales Tax

Shipping & Handling

Other

TOTAL

1,898.00$

5%

20.00$

15.00$

2,027.90$

686 Harrison Lane - Union City - CA - 94587 - 555 111 5533




### Definition of the prompt that you will use in order to extract the required information

In [28]:
# Change the prompt according to the PO layout you want to extract
base_prompt = '''

You are an AI assistant that helps extract information from purchase orders.

Extract the Purchase Order number, PO Status, the Supplier name and address, Customer Name and Address, the delivery date, order total and all items in the table.

Output the information in JSON format in the following structure:

{
 purchase_order:
 purchase_order_status:
 customer_name:
 customer_address:
 supplier_name:
 supplier_address:
 delivery_date:
 order_total:
 table:
[
 {
  item_name:
  item_code:
  quantity:
  item_price:
 discount:
 total:

 }
]
}

'''

### Call the Chat Completion APIs

In [29]:

headers = {
    "Content-Type": "application/json",
    "api-key": GPT4V_KEY,
}

# Payload for the request
payload = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "You are an AI assistant that extract information from an image that contains purchase order information. Output only plain text. Do not output markdown"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": base_prompt
        },
        {
          "type": "text",
          "text": extracted_text
        }
      ]
    }
  ],
  "temperature": 0,
  "top_p": 0.95,
  "max_tokens": 4096
}


# Send request
try:
    response = requests.post(GPT4V_ENDPOINT, headers=headers, json=payload)
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")


result = response.json()


### Now you have the results in JSON format and you can integrate it with any other service to continue with your automation

##### Try changing the prompts or instruction to process more fields such as Department, notes, etc.

In [32]:
result = response.json()

return_value = result["choices"][0]["message"]["content"]

print(return_value)


{
  "purchase_order": "679133",
  "purchase_order_status": "",
  "customer_name": "Robert Web",
  "customer_address": "RW Wines, 7310 Somerset St., Salinas - CA - 93905",
  "supplier_name": "John Smith",
  "supplier_address": "Cellar Selections, 686 Harrison Lane, Union City - CA - 94587",
  "delivery_date": "19/05/2024",
  "order_total": "2,027.90$",
  "table": [
    {
      "item_name": "Cabernet Sauvignon",
      "item_code": "2010",
      "quantity": "8",
      "item_price": "32.00$",
      "discount": "5%",
      "total": "243.20$"
    },
    {
      "item_name": "Cabernet Sauvignon",
      "item_code": "2002",
      "quantity": "4",
      "item_price": "45.00$",
      "discount": "5%",
      "total": "171.00$"
    },
    {
      "item_name": "Rose",
      "item_code": "2019",
      "quantity": "24",
      "item_price": "25.00$",
      "discount": "15%",
      "total": "510.00$"
    },
    {
      "item_name": "Syrah",
      "item_code": "2016",
      "quantity": "12",
      "item